In [ ]:
# Imports
from snowflake.snowpark.session import Session
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.preprocessing import MinMaxScaler, LabelEncoder, OrdinalEncoder
from snowflake.ml.modeling.metrics import r2_score, accuracy_score, precision_score, roc_auc_score, f1_score, recall_score
from snowflake.snowpark.functions import col, is_null, regexp_replace, when, lit
from snowflake.snowpark.types import StringType
from snowflake.snowpark.exceptions import SnowparkSQLException
import importlib, sys, json, os, logging


In [ ]:
CONNECTION_PARAMETERS = {
    "account": "ug94937.us-east4.gcp",
    "user":"ADITYASINGH",
    "password": os.environ.get('SF_Password'),
    "role": "ADITYASINGH",
    "database": "FIRST_DB",
    "warehouse": "FOSFOR_INSIGHT_WH",
    "schema": "PUBLIC"
}

In [ ]:
def create_stage(session, stage_name="demo"):
    try:
        session.sql(f"create or replace stage {stage_name}").collect()
        return f"@{stage_name}"
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

        
def get_session():
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex
  

def apply_data_cleansing(df):
    """
    Method handles null values in snowpark dataframe.
    :param:
    df: input dataframe
    :returns:
    df_cleaned: dataframe after null handling
    """
    
    